In [1]:
import torchsummary as summary
import onnx
import torch
import os
import sys
from pathlib import Path

from python.craftdet.models import *
from python.craftdet.models.utils import load_model


## Craftdet

In [4]:
# base_net = basenet.BaseNet()
# summary.summary(base_net, (3, 256, 256), device='cpu')

In [5]:
# print(base_net.features)

In [6]:
# # export basenet to onnx
# import torch


# torch_model = BaseNet(freeze=False, pretrained=False)

# torch_input = torch.randn(1, 3, 256, 256)
# onnx_program = torch.onnx.export(torch_model, torch_input, "models/basenet.onnx")


In [7]:
# onnx_model = onnx.load('models/basenet.onnx')

In [8]:
# onnx_model.graph.output

In [9]:
# sources = [
#     torch.rand([1024, 16, 16]),
#     torch.rand([512, 16, 16]),
#     torch.rand([512, 32, 32]),
#     torch.rand([256, 64, 64]),
#     torch.rand([128, 128, 128])
# ]

In [2]:
import torch 
basenet = basenet.BaseNet(pretrained=False, freeze=False)
craft_net = craftnet.CraftNet(basenet=basenet)

summary.summary(craft_net, (3, 512, 512), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]           1,792
       BatchNorm2d-2         [-1, 64, 512, 512]             128
              ReLU-3         [-1, 64, 512, 512]               0
            Conv2d-4         [-1, 64, 512, 512]          36,928
       BatchNorm2d-5         [-1, 64, 512, 512]             128
              ReLU-6         [-1, 64, 512, 512]               0
         MaxPool2d-7         [-1, 64, 256, 256]               0
            Conv2d-8        [-1, 128, 256, 256]          73,856
       BatchNorm2d-9        [-1, 128, 256, 256]             256
             ReLU-10        [-1, 128, 256, 256]               0
           Conv2d-11        [-1, 128, 256, 256]         147,584
      BatchNorm2d-12        [-1, 128, 256, 256]             256
             ReLU-13        [-1, 128, 256, 256]               0
        MaxPool2d-14        [-1, 128, 1

In [13]:
torch_input = torch.randn((1, 3, 512, 512), dtype=torch.float32)
craft_net.basenet = torch.jit.script(craft_net.basenet, torch_input)
craft_net_trace = torch.jit.script(craft_net, torch_input)
print(craft_net_trace.basenet.code)
print(craft_net_trace.code)

def forward(self,
    x: Tensor) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor]:
  slice5 = self.slice5
  slice4 = self.slice4
  slice3 = self.slice3
  slice2 = self.slice2
  slice1 = self.slice1
  _0 = (slice2).forward((slice1).forward(x, ), )
  _1, _2, = _0
  _3, _4, = (slice3).forward(_1, )
  _5, _6, = (slice4).forward(_3, )
  _7 = ((slice5).forward(_5, ), _5, _6, _4, _2)
  return _7

def forward(self,
    x: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = __torch__.torch.nn.functional.___torch_mangle_317.interpolate
  basenet = self.basenet
  sources = (basenet).forward(x, )
  y = torch.cat([(sources)[0], (sources)[1]], 1)
  upconv1 = self.upconv1
  y0 = (upconv1).forward(y, )
  _1 = torch.slice(torch.size((sources)[2]), 2)
  y1 = _0(y0, _1, None, "bilinear", False, None, False, )
  y2 = torch.cat([y1, (sources)[2]], 1)
  upconv2 = self.upconv2
  y3 = (upconv2).forward(y2, )
  _2 = torch.slice(torch.size((sources)[3]), 2)
  y4 = _0(y3, _2, None, "bilinear", False, None, False, )
  y5 = 

/home/hungnq/.conda/envs/craftdet/lib/python3.11/site-packages/torch/jit/_script.py:1277: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


In [5]:
use_cuda = torch.cuda.is_available()
IMAGE_SIZE = 512

In [6]:
craft: str = './weights/craft/mlt25k.pth'
craft_net = load_model(CraftNet(basenet), craft, use_cuda)
torch_input = torch.randn((1, 3, IMAGE_SIZE, IMAGE_SIZE), device='cuda' if use_cuda else 'cpu')
onnx_program = torch.onnx.export(craft_net, torch_input, "models/craftnet.onnx")
# onnx_program.save()

/home/hungnq/.conda/envs/craftdet/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


In [7]:
refiner: str = './weights/craft/refinerCTW1500.pth'
refine_net = load_model(RefineNet(), refiner, use_cuda)
torch_input = (
    torch.randn((1, IMAGE_SIZE, IMAGE_SIZE, 2), device='cuda' if use_cuda else 'cpu'),
    torch.randn((1, 32, IMAGE_SIZE, IMAGE_SIZE), device='cuda' if use_cuda else 'cpu')
)
onnx_program = torch.onnx.export(refine_net, torch_input, "models/retinet.onnx")

/home/hungnq/.conda/envs/craftdet/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


## OCR

In [2]:
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

config = Cfg.load_config_from_name('vgg_seq2seq')
config['weights'] = str(Path(os.getcwd() + '/weights/ocr/vgg_seq2seq.pth').expanduser().resolve())
config['device'] = 'cuda:0'
# ocr = Predictor(config)
save_ocr_path = Path(os.getcwd()+ "/prediction/ocr").expanduser().resolve()

In [3]:
config

{'vocab': 'aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ',
 'device': 'cuda:0',
 'seq_modeling': 'seq2seq',
 'transformer': {'encoder_hidden': 256,
  'decoder_hidden': 256,
  'img_channel': 256,
  'decoder_embedded': 256,
  'dropout': 0.1},
 'optimizer': {'max_lr': 0.001, 'pct_start': 0.1},
 'trainer': {'batch_size': 32,
  'print_every': 200,
  'valid_every': 4000,
  'iters': 100000,
  'export': './weights/transformerocr.pth',
  'checkpoint': './checkpoint/transformerocr_checkpoint.pth',
  'log': './train.log',
  'metrics': None},
 'dataset': {'name': 'data',
  'data_root': './img/',
  'train_annotation': 'annotation_train.txt',
  'valid_annotation': 'annotation_val_small.txt',
  'image_height': 32,
  'image_min_width': 32,
  'image_max_width': 512},
 'dataloader': {'num_workers': 3, 'pin_memory': True}

In [4]:
from vietocr.model.transformerocr import VietOCR
from vietocr.model.vocab import Vocab

vocab = Vocab(config['vocab'])
device = config['device']

model = VietOCR(len(vocab),
    config['backbone'],
    config['cnn'], 
    config['transformer'],
    config['seq_modeling']
)
model.load_state_dict(torch.load(config['weights'] , map_location=torch.device(config['device'])))

<All keys matched successfully>

In [35]:
import numpy as np

eos_token = 1
translated_sentence = [[0]*10]
char_probs = [[0]]
memory = (torch.zeros(10, 256), torch.zeros(1, 10, 512))
max_length = 0

while max_length <= 128 and not all(np.any(np.asarray(translated_sentence).T==eos_token, axis=1)):
    tgt_inp = torch.LongTensor(translated_sentence).to(device)
    print(tgt_inp.shape)
    output, memory = model.transformer.forward_decoder(tgt_inp, memory)
    print(output.shape)
    print(memory[0].shape)
    print(memory[1].shape)
    break
    max_length += 1

torch.Size([1, 10])
torch.Size([10, 1, 233])
torch.Size([10, 256])
torch.Size([1, 10, 512])


In [5]:
# model(torch.randn((1, 3, 256, 256)), rch.randint(0, 100, (1, 2)), torch.randint(0, 100, (1, 2)))

In [8]:
# CNN part
def convert_cnn_part(img, save_path, model, max_seq_length = 128, sos_token = 1, eos_token = 2): 
    with torch.no_grad(): 
        src = model.cnn(img)
        torch.onnx.export(model.cnn, 
                          img, 
                          save_path, 
                          export_params = True, 
                          do_constant_folding = True, 
                          verbose = True, 
                          input_names = ['IMAGE'], 
                          output_names = ['OUTPUT'], 
                          dynamic_axes = {'IMAGE': {0: 'batch'}, 'OUTPUT': {1: 'batch'}}) # channel output of cnn in the 2nd dimension
     
    return src

# Encoder part
def convert_encoder_part(model, src, save_path): 
    encoder_outputs, hidden = model.transformer.encoder(src) 
    torch.onnx.export(model.transformer.encoder, 
                      src, 
                      save_path, 
                      export_params = True, 
                      opset_version = 11, 
                      do_constant_folding = True, 
                      input_names  = ['src'], 
                      output_names = ['encoder_outputs', 'hidden'], 
                      dynamic_axes = {'src' : {0 : "channel_input"}, 
                                      'encoder_outputs' : {0 : 'channel_output'}}) 
    return hidden, encoder_outputs

# Decoder part
def convert_decoder_part(model, tgt, hidden, encoder_outputs, save_path):
    tgt = tgt[-1]
    print(tgt)
    torch.onnx.export(  model.transformer.decoder,
                        (tgt, hidden, encoder_outputs),
                        save_path,
                        export_params = True,
                        opset_version = 11,
                        do_constant_folding = True,
                        input_names = ['tgt', 'hidden', 'encoder_outputs'],
                        output_names = ['output', 'hidden_out', 'last'],
                        dynamic_axes = {'encoder_outputs' : {0 : "channel_input"}, 
                                        'last' : {0 : 'channel_output'}})


# Export model
img = torch.rand(1, 3, 512, 512)   # input tensor of torch model (N x C x H x W)
src = convert_cnn_part(img, 'models/ocr_cnn/1/model.onnx', model)

hidden, encoder_outputs = convert_encoder_part(model, src, 'models/ocr_encoder/1/model.onnx')

device = img.device
tgt = torch.LongTensor([[1] * len(img)]).to(device)
convert_decoder_part(model, tgt, hidden, encoder_outputs, 'models/ocr_decoder/1/model.onnx')

Exported graph: graph(%IMAGE : Float(*, 3, 512, 512, strides=[786432, 262144, 512, 1], requires_grad=0, device=cpu),
      %model.last_conv_1x1.weight : Float(256, 512, 1, 1, strides=[512, 1, 1, 1], requires_grad=1, device=cpu),
      %model.last_conv_1x1.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %onnx::Conv_180 : Float(64, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_181 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_183 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_184 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_186 : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_187 : Float(128, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_189 : Float(128, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_190 : Float(128, strides=[1], requires_

/home/hungnq/.conda/envs/craftdet/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:4662: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
/home/hungnq/.conda/envs/craftdet/lib/python3.11/site-packages/vietocr/model/seqmodel/seq2seq.py:93: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert (output == hidden).all()


tensor([1])


In [20]:
summary.summary(model, (3, 512, 512), device='cpu')

TypeError: VietOCR.forward() missing 2 required positional arguments: 'tgt_input' and 'tgt_key_padding_mask'

## Script testing tracing
code: https://pytorch.org/tutorials//beginner/Intro_to_TorchScript_tutorial.html

In [8]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h
    
class MyCell2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h


In [10]:
x, h = torch.rand(3, 4), torch.rand(3, 4)

In [11]:
my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell.dg.code)
print(traced_cell.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  return torch.neg(argument_1)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  _1 = torch.tanh(_0)
  return (_1, _1)



/tmp/ipykernel_964/1475766357.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


In [16]:
my_cell2 = MyCell2()
traced_cell2 = torch.jit.trace(my_cell2, (x, h))
print(traced_cell2.dg.code)
print(traced_cell2.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  return torch.neg(argument_1)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  _1 = torch.tanh(_0)
  return (_1, _1)



/tmp/ipykernel_964/1475766357.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


In [17]:
script_cell2 = torch.jit.script(my_cell2, (x, h))
print(script_cell2.dg.code)
print(script_cell2.code)

def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.sum(x), 0)):
    _0 = x
  else:
    _0 = torch.neg(x)
  return _0

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  new_h = torch.tanh(_0)
  return (new_h, new_h)



/home/hungnq/.conda/envs/craftdet/lib/python3.11/site-packages/torch/jit/_script.py:1277: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(
